In [19]:
from mask_utils.utils import *
from shutil import copyfile
from tqdm.notebook import tqdm

In [6]:
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
          (0, 255, 255), (255, 0, 255), (0, 0, 0), (255, 255, 255)]
random.seed(2021)
np.random.seed(2021)

In [3]:
with open('../MERGE_OCEAN/Annotations/merge_ocean_train.json', 'r') as json_file:
    train_anno = json.load(json_file)
json_file.close()
list_images = train_anno['images']
list_names = [item['name'] for item in train_anno['categories']]

In [ ]:
n = 1
p = 0.5
MAX_INDEX = len(list_images)
np.random.seed(2021)
s = np.random.binomial(n, p, MAX_INDEX)

transformed_annotations = dict()
transformed_annotations['categories'] = deepcopy(train_anno['categories'])
transformed_annotations['images'] = []
transformed_annotations['annotations'] = []
count_img = 0
count_ins = 0
    
for i in tqdm(range(MAX_INDEX)):
    
    if s[i] == 1:
        while True:
            x = random.randint(0, MAX_INDEX-1)
            if x != i:
                break
                
        transformed_img, polygon, transformed_imganno, transformed_anno = mixup_image(list_images[i], 
                                                                                      list_images[x], 
                                                                                      count_img, 
                                                                                      '../MERGE_OCEAN/Train/RAND_MIXUP', 
                                                                                      train_anno)

        cv2.imwrite(transformed_imganno['file_name'], transformed_img)
        
    else:
        transformed_anno = []
        img_id = list_images[i]['id']
        for ann in train_anno['annotations']:
            image_id = ann['image_id']
            if image_id == img_id:
                tmp_ann = deepcopy(ann)
                tmp_ann['image_id'] = count_img
                transformed_anno.append(tmp_ann)
                
        transformed_imganno = deepcopy(list_images[i])
        transformed_imganno['id'] = count_img
        
    count_img += 1
    transformed_annotations['images'].append(transformed_imganno)

    for j in range(len(transformed_anno)):
        transformed_anno[j]['id'] = count_ins
        count_ins += 1

    transformed_annotations['annotations'].extend(transformed_anno)    

In [11]:
with open('../MERGE_OCEAN/Annotations/rand_mixup.json', 'w') as json_file:
    json.dump(transformed_annotations, json_file)
json_file.close()